In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Импорты. Потом почистить.

In [2]:
# импорт библиотек

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pandas_profiling
from pandas_profiling import ProfileReport
import os
import sys
import zipfile
import PIL
from PIL import ImageOps, ImageFilter
import re
import ast
from datetime import time, date, datetime
import random

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.style.use('seaborn-paper')

from optuna import *

from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_selection import SelectKBest, chi2, f_classif, f_regression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MinMaxScaler, StandardScaler, RobustScaler
from sklearn.impute import KNNImputer

from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.metrics import mean_squared_error, f1_score, accuracy_score, roc_curve, roc_auc_score
from sklearn import metrics
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import VotingRegressor
from tqdm.notebook import tqdm
from catboost import CatBoostRegressor, Pool, cv

#image augmentation
#import albumentations as A
#from ImageDataAugmentor.image_data_augmentor import *

# keras
import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow.keras.layers as L
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.applications.efficientnet import EfficientNetB3
from tensorflow.keras.regularizers import l2
from tensorflow.keras.losses import MeanAbsolutePercentageError

from transformers import AutoTokenizer, TFAutoModelForSequenceClassification, DataCollatorWithPadding

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')


pd.set_option('display.max_columns', 550) # больше колонок

os.environ['PYTHONHASHSEED']=str(42)

In [3]:
RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)
ITERATIONS = 200

In [4]:
DIR_DATA  = '../input/diploma/'

In [5]:
data_d = pd.read_csv(DIR_DATA+'data.csv')

# ВСПОМОГАТЕЛЬНЫЕ ФУНКЦИИ

Вспомогательные функции для обработки признаков и приведения их к численному виду.

In [6]:
def extract_several_digits_comma(stroka,first_digit,len_s,i):
    if i==len_s and not stroka[i].isdigit():
        return (first_digit)
    elif i==len_s and stroka[i].isdigit():
        
        return (10*first_digit+int(stroka[i]))

    else:
        if stroka[i] == ',':
            return extract_several_digits_comma(stroka, first_digit, len_s,i+1)
        elif not stroka[i].isdigit():
            return (first_digit)
        
        
        else:
            return extract_several_digits_comma(stroka,(10*first_digit+int(stroka[i])),len_s,i+1)
        
        
def extract_num_recurr_comma(stroka,len_s,i):

    if i==len_s and not stroka[i].isdigit():
       
        return 0
   
    elif i<len_s and stroka[i].isdigit():
        
        return extract_several_digits_comma(stroka,int(stroka[i]),len_s,i+1)
    
    else: return(extract_num_recurr_comma(stroka,len_s,i+1))  
    
    
# основная функция для нахождения числа. На вход подается только строка, которую нужно обработать.

def extract_num_comma(stroka):

    len_s=len(str(stroka))-1
 
    return extract_num_recurr_comma(str(stroka),len_s,0)
    
    

In [7]:
def extract_several_digits(stroka,first_digit,len_s,i):
    if i==len_s and not stroka[i].isdigit():
        return (first_digit)
    elif i==len_s and stroka[i].isdigit():
        
        return (10*first_digit+int(stroka[i]))
    else:
        if not stroka[i].isdigit():
            return (first_digit)
        else:
            return extract_several_digits(stroka,(10*first_digit+int(stroka[i])),len_s,i+1)
    

def extract_num_recurr(stroka,len_s,i):

    if i==len_s and not stroka[i].isdigit():
       
        return 0
   
    elif i<len_s and stroka[i].isdigit():
        
        return extract_several_digits(stroka,int(stroka[i]),len_s,i+1)
    
    else: return(extract_num_recurr(stroka,len_s,i+1))
        
# основная функция для нахождения числа. На вход подается только строка, которую нужно обработать.
def extract_num(stroka):

    len_s=len(str(stroka))-1
 
    return extract_num_recurr(str(stroka),len_s,0)

Вспомогательная функция stirng_part со вложенной string_part_extract для выделения символьной части.

Нужно, чтобы определить различные обозначения численного значения и неверные обозначения.

In [8]:
def string_part_extract(string, len_s, i, res_str):
    if i<len_s:
      
        if string[i].isdigit(): return string_part_extract(string, len_s, i+1, res_str)
        else:
    
            return string_part_extract(string, len_s, i+1, res_str+string[i])
    elif i==len_s:
    
        if string[i].isdigit(): 
           
            return res_str
        
        else:
       
            return (res_str+string[i])
    
    
    
    
def stirng_part(string):
    len_s = len(str(string))-1
    pus = ""
    return string_part_extract(str(string), len_s, 0, pus)

Вспомогательная функция subsrting_beds(source_string) проверяет отсутствие определенных слов в текстовой части описания признака beds.

In [9]:
def subsrting_beds(source_string):
    list_of_strings = ['Bd','Bedrooms','Beds','Bed','bd','beds','bed','bedrooms']
    for substring in list_of_strings:
        if substring in source_string:
            return False
    return True

# Вспомогательная функция для выбора числа в признаке спальни.
(Не самая элегантная функция, если будет время еще подумаю.)

In [10]:
def otbor(stroka,i):
    if i == len(stroka): return(int(stroka))
    elif stroka[i] == ' ':
            return (int(stroka[:i]))
    else: return otbor(stroka,i+1)
    

# Вспомогательная функция для получения численного значения оборудования в ванной. Если значение некорректно, выдает ноль.

In [11]:
def extract_num_recurr_bath(stroka,first_digit,len_s,i):
    if i == len_s and stroka[i].isdigit():
        return (float(first_digit + str(stroka[i])))
    elif i < len_s and (stroka[i].isdigit() or stroka[i] == '.'):
        return extract_num_recurr_bath(stroka, (first_digit + str(stroka[i])), len_s, i+1)
    
    else: return(float(first_digit))
                        
def extract_num_bath(stroka,len_s,i):
  
    if i == len_s and not stroka[i].isdigit():
        return 0
    elif i == len_s and stroka[i].isdigit():
        return float(stroka[i])
    elif i < len_s and stroka[i].isdigit():
        return extract_num_recurr_bath(stroka,str(stroka[i]),len_s,i+1)
    else: return extract_num_bath(stroka,len_s,i+1)

def main_extract_num_bath(stroka):   
    len_s=len(stroka)-1
    
    return extract_num_bath(stroka,len_s,0)

# Вспомогательная функция для получения численного значения площади. Если значение некорректно, выдает ноль.

In [12]:
def extract_num_recurr_sqft(stroka,first_digit,len_s,i):
    if i == len_s and stroka[i].isdigit():
        return (float(first_digit + str(stroka[i])))
    elif i < len_s and (stroka[i].isdigit() or stroka[i] == '.'):
        return extract_num_recurr_sqft(stroka, (first_digit + str(stroka[i])), len_s, i+1)
    elif i < len_s and stroka[i] == ',':
        return extract_num_recurr_sqft(stroka, first_digit, len_s, i+1)
    else: return(float(first_digit))
                        
def extract_num_sqft(stroka,len_s,i):
  
    if i == len_s and not stroka[i].isdigit():
        return 0
    elif i == len_s and stroka[i].isdigit():
        return float(stroka[i])
    elif i < len_s and stroka[i].isdigit():
        return extract_num_recurr_sqft(stroka,str(stroka[i]),len_s,i+1)
    else: return extract_num_sqft(stroka,len_s,i+1)

def main_extract_num_sqft(stroka):   
    len_s=len(stroka)-1
    
    return extract_num_sqft(stroka,len_s,0)

# Вспомогательная функция для извлечения числа из записи доллара.

In [13]:
def dollar_clean(string):
    i=0
    string_interm=re.findall('[0-9]+', string)
    
    if string_interm==[]:

        return 0
    else:
        dollar_res=''
        while i<=(len(string_interm)-1):
            dollar_res=dollar_res + string_interm[i]
            i=i+1
        return (float(dollar_res))


# Вспомогательная функция для приведения признака числа каминов к численному виду.

In [14]:
def fireplaces_def(stroka):
    if stroka == 'yes': return 1
    elif stroka == 'Yes': return 1
    elif stroka == '1': return 1
    elif stroka == '2': return 2
    else: return 0

# Вспомогательные функции расчета метрик.

In [15]:
# Функция расчета ошибки
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

In [89]:
# Функция расчета ошибки для логарифмированной целевой переменнной
def mape_log(y_true, y_pred):
    return np.mean(np.abs((np.exp(y_pred)-np.exp(y_true))/np.exp(y_true)))

In [17]:
def print_regression_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    print(f'MSE = {mse:.2f}, RMSE = {rmse:.2f}')

# Общие сведения, задачи, примерный план работы.

In [18]:
data_d.info()

Бизнес-задача. Назначение "правильной" цены, исходя из параметров дома. То есть, максимально близкой к продажной.
Бизнес-метрика. Пока не продумала.

1. + Что такое статус?

    1.1. + Привести к численному значению. Видимо, будет много выбросов. Посмотреть корреляцию.
    
2. + Насколько важен private pool? Посмотреть распределение значение и проверить корреляцию с ценой - вдруг будет очень существенный скачок.

    2.1. В принципе, строк мало, можно пренебречь. Либо проверить количество значений значительно больше средней цены, после приведения цены к численному значению.
    
    #удалено из предварительной выборки
    
3. + Тип собственности: преобразовать вручную. Посмотреть, какие есть значения.

    3.1. + Подпрограмма поиска разных вариантов названия: single family home, может есть другие.
    
    3.2. + Приведение к численным значениям.
    
4. С точки зрения здравого смысле должна быть корреляция между улицей и городом. Возможны проблемы с сочетанием популярности улицы - непопулярности города и наоборот. Что с этим делать пока не знаю.

Объединила штат и город в единый признак, улицу удалила из предварительной выборки. С точки зрения здравого смысла, должна влиять на цену, как обрабатывать пока непонятно.

5. + Преобразовать площадь в численное значение.
6. Нужен ли индекс?  # удалено
7. + Ванны: выделить численное значение. Как быть с тремя с половиной ваннами? (Скорее всего, уйдет как выброс.)
8. homefacts - потом NLP
9. + Насколько важен камин? Посмотреть количество значений и нет ли существенного скачка в цене.
10. Город - см. улица.
11. Школы: посмотреть корреляцию. Преобразовывать это будет сложно.

 Пока удалила из предварительной выборки. Пока нужно думать. С точки зрения здравого смысла, на цену должно влиять.

12. + Площадь привести к численному выражению.
13. + Спальни: привести к численному выражению.

    13.1. + Добавить проверку на наличие более чем двузначных чисел. Посмотреть, сколько всего.
    
    13.2. + Заменить пропуски каким-нибудь числом. Посмотреть, сколько всего.
    
    13.3. + Если мало, выкинуть пропуски и некорректно заполненные. Если много, подумать, чем заполнить пропуски. С точки зрения здравого смысла, признак важный.
    
14. Штат: та же фигня, что с городом и улицей.
15. + Stories: преобразовать к численному, посмотреть корреляцию.
16. mls_id и MLSID: похоже, одно и то же. Посмотреть распределение значений mls_id, скорее всего, можно будет просто выкинуть. MLSID, наверное, тоже.  # удалено
17. + Цена: преобразовать к численному выражению.

Дополнение по описанию и школам от 04 09.

18. Из признака "факты" можно извлечь дополнительные признаки (например, гараж).

    18.1. В "фактах" есть площадь. Ее можно извлечь и заполнить пропуски, если есть сочетание: площадь в "фактах" - нет записи в площади. Хотя что-то мне подсказывает, что из "фактов" площадь и брали. Ну, возможно.
    
    18.1. По обработке записи фактов. Выглядит как будто просто: получается список словарей, из которых легко извлекается значение по ключу. Запуталась при написании программы.
    
19. По признаку школы.
    
    19.1. Выделить подпризнаки: рейтинг школы, расстояние, grade, название.
    
    19.2. Все подпризнаки выделить в самостоятельные признаки, посмотреть корреляцию. Скорее всего, с названием будет низкая корреляция. Сложности: нужно как-то хитро обработать исходный текст как строку, потом добавить колонки в датафрейм. А там еще может быть сложность, потому что не равное число данных по каждому подпризнаку. 
    

Удалено, потому что громоздко.

In [19]:
# Быстрый EDA 
#profile = ProfileReport(data_d, title='Dataset Pandas Profiling Report', explorative = True)

In [20]:
#profile

# Спальни - приведение к численному виду.

In [21]:
data_d.head()

In [22]:
data_d['beds'].unique()

In [23]:
data_d = data_d.drop(np.where(data_d['beds'] == '# Bedrooms 1st Floor')[0])

In [24]:
data_d.beds = data_d.beds.apply(lambda x: 0 if subsrting_beds(stirng_part(x)) else x)
data_d.beds = data_d.beds.apply(lambda x: 0 if 'Bath' in str(x) else x)
data_d.beds = data_d.beds.apply(lambda x: 0 if x == '1-2 Beds' or x == '-- bd' else x)
data_d['beds'].unique()                        
data_d['beds'].value_counts() 

In [25]:
data_d.beds = data_d.beds.apply(lambda x: otbor(str(x),0))

In [26]:
data_d['beds'].unique()                        


# Ванны - приведение к численному виду.

In [27]:
data_d['baths'].unique()   

In [28]:
data_d.baths = data_d.baths.apply(lambda x: main_extract_num_bath(str(x)))

In [29]:
data_d['baths'].unique()   

 # Камин

In [30]:
data_d['fireplace'].value_counts(3) 

In [31]:
data_d.fireplace = data_d.fireplace.apply(lambda x: fireplaces_def(str(x)))


In [32]:
data_d['fireplace'].unique()

# Бассейн. Анализ.

In [33]:
data_d['private pool'].value_counts()

In [34]:
data_d.loc[data_d['private pool'] == 'Yes']

Вывод: число домов с бассейнами незначительно, разброс цен для домов с бассейнами большой, явного влияния на цену нет. Можно не рассматривать.

 # propertyType

In [35]:
dd = data_d['propertyType'].value_counts(normalize=True)

In [36]:
data_d.propertyType = data_d.propertyType.apply(lambda x: 'sfh' if x in ['single-family home','Single Family','Single Family Home'] else x)

In [37]:
data_d['propertyType'].value_counts(normalize=True)

In [38]:
#pd.get_dummies(data_d.propertyType).head()

In [39]:
#propertyT_feature = ['propertyType']
#data_d = pd.get_dummies(data_d, columns = propertyT_feature, dummy_na=False)

#data_d.propertyType = pd.get_dummies(data_d.propertyType)
#df_output = pd.get_dummies(df_output, columns=categorical_features, dummy_na=False)

  # One-Hot Encoding: в pandas есть готовая функция - get_dummies.
    #df_output = pd.get_dummies(df_output, columns=categorical_features, dummy_na=False)
    #categorical_features = ['bodyType', 'brand', 'color', 'engineDisplacement', 'enginePower', 'fuelType', 'model_info', 'name',
  #'numberOfDoors', 'vehicleTransmission', 'Владельцы', 'Владение', 'ПТС', 'Привод', 'Руль']


In [40]:
data_d['propertyType'] = data_d['propertyType'].astype('category')
data_d['propertyType_cat'] = data_d['propertyType'].cat.codes

 # площадь

In [41]:
data_d.sqft = data_d.sqft.apply(lambda x: main_extract_num_sqft(str(x)))

In [42]:
data_d.sqft = data_d.sqft.apply(lambda x: data_d['sqft'].median() if x == 0 else x)

In [43]:
data_d['sqft'].value_counts()

In [44]:
data_d.target = data_d.target.apply(lambda x: dollar_clean(str(x)))

In [45]:
data_d['sity_state']=data_d['city'].astype(str) + " " +data_d['state'].astype(str)


In [46]:
data_d['sity_state'] = data_d['sity_state'].astype('category')
data_d['sity_state_cat'] = data_d['sity_state'].cat.codes

 # Stories (надеюсь, это этажи)

In [47]:
data_d['stories'].value_counts()

In [48]:
data_d.stories = data_d.stories.apply(lambda x: main_extract_num_bath(str(x)))


In [49]:
data_d['stories'].value_counts()

 # Status

In [50]:
data_d['status'].value_counts()

In [51]:
data_d.status = data_d.status.apply(lambda x: 'for_sale' if x in ['for sale','For sale'] else x)

Я бы предположила, что active - это на самом деле тоже for sale, но на всякий случай пока оставлю.

In [52]:
data_d['status'].value_counts()

In [53]:
data_d['status'] = data_d['status'].astype('category')
data_d['status_cat'] = data_d['status'].cat.codes

# Выбор предварительного датафрейма для модели

In [54]:
data_d.info()

In [55]:
data_d_preliminary=data_d[['status_cat','propertyType_cat','baths','fireplace','sqft','beds','stories','sity_state_cat','target']]

In [56]:
data_d_preliminary

In [57]:
data_d_preliminary.isnull().sum()

# Визуализация

In [58]:
plt.figure(figsize=(18, 10))
ax = sns.heatmap(data_d_preliminary.corr().abs(), vmin=0, vmax=1, annot = True)
i, k = ax.get_ylim()
ax.set_ylim(i+0.5, k-0.5)

In [59]:
#  подготовка данных для графиков
x1 = data_d_preliminary['propertyType_cat']
x2 = data_d_preliminary['status_cat']
x3 = data_d_preliminary['baths']
x4 = data_d_preliminary['fireplace']
x5 = data_d_preliminary['sqft']
x6 = data_d_preliminary['beds']
x7 = data_d_preliminary['stories']
x8 = data_d_preliminary['sity_state_cat']

y = data_d_preliminary['target']

In [60]:
plt.figure(figsize = (10,6))
plt.scatter(x2, y)
 
# добавим подписи
plt.xlabel('Статус продажи', fontsize = 15)
plt.ylabel('Цена', fontsize = 15)
plt.title('Зависимость цены от статуса продажи', fontsize = 18)

In [61]:
plt.figure(figsize = (10,6))
plt.scatter(x1, y)
 
# добавим подписи
plt.xlabel('Тип недвижимости', fontsize = 15)
plt.ylabel('Цена', fontsize = 15)
plt.title('Зависимость цены от типа недвижимости', fontsize = 18)

In [62]:
plt.figure(figsize = (10,6))
plt.scatter(x4, y)
 
# добавим подписи
plt.xlabel('Камин', fontsize = 15)
plt.ylabel('Цена', fontsize = 15)
plt.title('Зависимость цены от наличия камина', fontsize = 18)

In [63]:
plt.figure(figsize = (10,6))
plt.scatter(x5, y)
 
# добавим подписи
plt.xlabel('Площадь', fontsize = 15)
plt.ylabel('Цена', fontsize = 15)
plt.title('Зависимость цены от площади', fontsize = 18)

In [64]:
plt.figure(figsize = (10,6))
plt.scatter(x6, y)
 
# добавим подписи
plt.xlabel('Количество комнат', fontsize = 15)
plt.ylabel('Цена', fontsize = 15)
plt.title('Зависимость цены от количества комнат', fontsize = 18)

In [65]:
plt.figure(figsize = (10,6))
plt.scatter(x7, y)
 
# добавим подписи
plt.xlabel('Этаж', fontsize = 15)
plt.ylabel('Цена', fontsize = 15)
plt.title('Зависимость цены от этажа', fontsize = 18)

In [66]:
plt.figure(figsize = (10,6))
plt.scatter(x8, y)
 
# добавим подписи
plt.xlabel('Город', fontsize = 15)
plt.ylabel('Цена', fontsize = 15)
plt.title('Зависимость цены от города', fontsize = 18)

In [67]:
#data_d_preliminary=data_d[['status_cat','propertyType_cat','baths','fireplace','sqft','beds','stories','sity_state_cat','target']]
sns.boxplot(data_d_preliminary['status_cat'])

In [68]:
sns.boxplot(data_d_preliminary['propertyType_cat'])

In [69]:
data_d_preliminary['propertyType_cat'] = data_d_preliminary[(data_d_preliminary['propertyType_cat'] > 1270)]['propertyType_cat']
sns.boxplot(data_d_preliminary['propertyType_cat'])

In [70]:
data_d_preliminary['baths'] = data_d_preliminary[(data_d_preliminary['baths'] < 10)]['baths']
sns.boxplot(data_d_preliminary['baths'])

In [71]:
sns.boxplot(data_d_preliminary['fireplace'])

In [72]:
sns.boxplot(data_d_preliminary['sqft'])

In [73]:
sns.boxplot(data_d_preliminary['beds'])

In [74]:
data_d_preliminary['beds'] = data_d_preliminary[(data_d_preliminary['beds'] < 7)]['beds']
sns.boxplot(data_d_preliminary['beds'])

In [75]:
sns.boxplot(data_d_preliminary['stories'])

In [76]:
data_d_preliminary['stories'] = data_d_preliminary[(data_d_preliminary['stories'] < 5)]['stories']
sns.boxplot(data_d_preliminary['stories'])

In [77]:
sns.boxplot(data_d_preliminary['sity_state_cat'])

In [78]:

data_d_preliminary['sity_state_cat'] = data_d_preliminary[(data_d_preliminary['sity_state_cat'] < 500 ) | (data_d_preliminary['sity_state_cat'] > 1700)]['sity_state_cat']
sns.boxplot(data_d_preliminary['sity_state_cat'])

377185

In [79]:
data_d_preliminary.isnull().sum()

In [80]:
values = {'propertyType_cat':1275,'sity_state_cat':0}
data_d_preliminary = data_d_preliminary.fillna(value=values)

In [81]:
data_d_preliminary = data_d_preliminary.dropna()

In [82]:
data_d_preliminary.isnull().sum()

In [83]:
plt.figure(figsize=(18, 10))
ax = sns.heatmap(data_d_preliminary.corr().abs(), vmin=0, vmax=1, annot = True)
i, k = ax.get_ylim()
ax.set_ylim(i+0.5, k-0.5)

> # Грубая модель

In [84]:
X = data_d_preliminary[['status_cat','propertyType_cat','baths','fireplace','sqft','beds','stories','sity_state_cat']]
y = data_d_preliminary[['target']]

X1 = data_d_preliminary[['sity_state_cat','baths','beds']]

In [85]:
X_train, X_test, y_train, y_test = train_test_split(X1, y, 
                                                    train_size=0.67, 
                                                    random_state=RANDOM_SEED)

In [86]:
extra_tree = ExtraTreesRegressor(n_estimators = 400, 
                                 n_jobs=-1, random_state=RANDOM_SEED)
extra_tree.fit(X_train, y_train)
predict_extra_tree = extra_tree.predict(X_test)

# оцениваем точность
print((mape(y_test, predict_extra_tree)))
#print((mape_log(y_test, predict_extra_tree)))
#print(f"MAPE: {(mape_log(y_test, predict_extra_tree))*100:0.2f}%")


In [87]:
catboost = CatBoostRegressor(iterations = 1000,
                          learning_rate = 0.5,
                          random_seed = RANDOM_SEED,
                          eval_metric='RMSE',
                          depth = 10,
                          l2_leaf_reg = 1)
catboost.fit(X_train, y_train,
         
         eval_set=(X_test, y_test),
         verbose_eval=100,
         use_best_model=True,
         plot=True
         )

In [90]:
predict_catboost = catboost.predict(X_test)

print(f"MAPE: {(mape_log(y_test, predict_catboost))*100:0.2f}%")

In [91]:
print_regression_metrics(y_test, predict_catboost)

In [92]:
model = LinearRegression()


In [93]:
model.fit(X_train, y_train)

In [94]:
y_pred = model.predict(X_test)


In [95]:
from sklearn import metrics

In [96]:
print('Root Mean Squared Error (RMSE):', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))


In [97]:
print('R2:', np.round(metrics.r2_score(y_test, y_pred), 2))